# Vector Modesolver with Substrate Loss (Leaky Mode, with PML)

This example demonstrates how to include perfectly matched layers (PMLs) at the edges of the computational window. The waveguide geometry is the same as in `basic_semivector.ipynb`, except that we now include a substrate layer with the same refractive index as the core. In this configuration, the modesolver returns a complex propagation constant, whose imaginary part represents the radiation loss into the substrate.

The PML is implemented using the stretchmesh routine with a complex stretching factor, corresponding to complex coordinate stretching.

In [ ]:
# Enable automatic reloading of modules (IPython only)
%reload_ext autoreload
%autoreload 2 

# load required modules
import modesolver as mode
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.collections import LineCollection

In [ ]:
# Refractive indices:
ns = 3.44;         # Substrate
n1 = 3.34          # Lower cladding
n2 = 3.44          # Core
n3 = 1.00          # Upper cladding (air)

# Vertical dimensions:
hs = 4.0;          # Substrate
h1 = 1.5;          # Lower cladding
h2 = 1.3           # Core thickness
h3 = 0.5           # Upper cladding
rh = 1.1           # Ridge height

# Horizontal dimensions:
rw = 1.0           # Ridge half-width
side = 1.5         # Space on side

# Grid size:
dx = 0.0125        # grid size (horizontal)
dy = 0.0125        # grid size (vertical)

# Build waveguide mesh (eps)
x,y,xc,yc,nx,ny,eps,edges = mode.waveguidemesh([ns,n1,n2,n3],[hs,h1,h2,h3],rh,rw,side,dx,dy,return_edges=True)

In [ ]:
# Complex coordinate stretching on south and west boundaries
x,y,xc,yc,dx,dy = mode.stretchmesh(x,y,[0,160,20,0],1+2j);

In [ ]:
wavelength = 1.55  # vacuum wavelength
nmodes = 1         # number of modes to compute

# NOTE:  when analyzing leaky modes, it is important that you provide a good 
# guess, usually by solving the mode of an equivalent non-leaky structure first:

guess = 3.388688    # this was the TE refractive index calculated without substrate

# Consider the quasi-TE mode:
neff, Hx, Hy, Hz = mode.wgmodes(wavelength,guess,nmodes,dx,dy,eps=eps,boundary='000A')

print(f"Real[nte] = {neff.real:.6f}")
print(f"Imag[nte] = {neff.imag:.6e}")

alpha = -20000 * neff.imag * 2 * np.pi / wavelength   # cm^-1
dBpercm = 10 * alpha / np.log(10)

print(f"Attenuation = {dBpercm:7.5f} dB/cm")

In [ ]:
# Plot TE mode fields
levels = np.arange(-45,0,3)
figTE, axHy = plt.subplots()

xr = x.real
yr = y.real

axHy.pcolormesh(xr,yr,abs(Hy.real),cmap='jet',vmax=1)
axHy.contour(xr,yr,20*np.log10(abs(Hy.real)),levels,colors='white',negative_linestyles='solid', linewidths=1)
axHy.set_aspect('equal')
axHy.axis([min(xr), max(xr), min(yr), max(yr)])
axHy.set_xlabel('x')
axHy.set_ylabel('y')
axHy.set_title(f'Hy (TE Mode)\n$ n_{{eff}} = {neff:.6f} $')
lHy = LineCollection(edges, colors='black', linewidths=1)
axHy.add_collection(lHy)

plt.show()